In [6]:
import numpy as np

In [7]:
# Physical constants
pi = 3.14159265358979324 # Pi
h = 6.626070040E-34 # Planck's constant (Js)
k_b = 1.38064852E-23 # Boltzmann constant (J/K)
c = 299792458.0 # Speed of light (m/s)
mu_0 = 4.0*pi * 1E-7 # Vacuum permeability (H/m)
epsilon_0 = 1.0/(mu_0 * c**2) # Vacuum permittivity (F/m)
m_0 = 9.10938356E-31 # Free electron mass (kg)
q_0 = -1.6021766208E-19 # Standard charge (C)

In [13]:
# Scale factors
time_scale = 1.0E-12
length_scale = 1.0E-9

# System parameters
d = 1.0E6*length_scale # Gap spacing (nm)
V_0 = 1.0E3 # Voltage over gap (V)
delta_t = 0.25E-3*time_scale # Time step (ps)

In [14]:
def Set_Init():
    # Initial position, velocity and acceleration
    r = np.array([0.0, 0.0, 1.0])*length_scale
    v = np.array([0.0, 0.0, 0.0])
    #a = np.array([0.0, 0.0, -1.0*V_0*q_0/(m_0*d)])
    a = np.array([0.0, 0.0, 0.0])
    return r, v, a

In [15]:
# Theoretical value without image charge effects
r, v, a = Set_Init()
time_exp = np.sqrt(-2.0*d*(d - r[2])*m_0/(q_0*V_0))
time_exp_int = time_exp / delta_t

print(time_exp)
print(time_exp_int)
steps_exp = int( np.ceil(time_exp_int) )
print(steps_exp)

1.0663605747082954e-10
426544.2298833181
426545


In [16]:
E = -1.0*V_0/d
pre_fac_c = q_0**2/(4*pi*m_0*epsilon_0)
div_fac_c = 1.0e-7 * c**2 / epsilon_0

# Acceleration without image charge effects
def Update_a():
    F = E*q_0
    return np.array([0.0, 0.0, F/m_0])

# Acceleration with image charge effects
def Update_a_ic():
    # Acceleration due to electric field
    a_E = np.array([0.0, 0.0, E*q_0/m_0])
    
    # Acceleration due to image charge above
    r_a = r.copy()
    r_a[2] = 2.0*d - r_a[2]
    a_a = -1.0*pre_fac_c * (r - r_a) / np.linalg.norm(r - r_a)**3
    
     # Acceleration due to image charge below
    r_b = r.copy()
    r_b[2] = -1.0*r_b[2]
    a_b = -1.0*pre_fac_c * (r - r_b) / np.linalg.norm(r - r_b)**3
    
    #a_ic = np.array([0.0, 0.0, q_0**2*div_fac_c*0.25*( 1.0/(d - r[2])**2 - 1.0/(r[2])**2 )]) / m_0
    
    #a_test = a_a + a_b
    #if (abs((a_test[2]-a_ic[2])/a_test[2]) > 0.01):
    #    print('Check!!!')
    
    #print(abs((a_test[2]-a_ic[2])))
    #print(a_test[2])
    #print(a_ic[2])
    
    #return a_E + a_ic
    return a_E + a_a + a_b

In [12]:
r, v, a = Set_Init()

# Simplified Velocity Verlet algorithm
for i in range(steps_exp+1000):
    r = r + v*delta_t + 0.5*a*delta_t**2
    a_new = Update_a_ic()
    v = v + 0.5*(a + a_new)*delta_t
    a = a_new
    if (r[2] > d):
        print('Above d')
        steps_res = i+1
        print(i+1) # range() starts from 0 so add +1
        break
    if (r[2] < 0.0):
        print('Below 0.0')
        steps_res = i+1
        print(i+1)
        break
    
print(r/1.0E-9)
steps_res = steps_exp - 2
diff_res = abs(steps_exp - steps_res)/steps_exp
print(diff_res*100)

Above d
2
[0.         0.         7.87742582]
200.0
